In [ ]:
import tkinter as tk
import copy

PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ''


class MiniBoard:
    def __init__(self):
        self.board = [[EMPTY for _ in range(3)] for _ in range(3)]
        self.winner = None

    def make_move(self, row, col, player):
        if self.board[row][col] == EMPTY and not self.winner:
            self.board[row][col] = player
            self.check_winner()
            return True
        return False

    def check_winner(self):
        lines = self.board + [list(col) for col in zip(*self.board)]
        lines.append([self.board[i][i] for i in range(3)])
        lines.append([self.board[i][2 - i] for i in range(3)])
        for line in lines:
            if EMPTY not in line and len(set(line)) > 1:
                self.winner = line[0]
                return
        if all(cell != EMPTY for row in self.board for cell in row):
            self.winner = 'Tie'

    def is_full(self):
        return all(cell != EMPTY for row in self.board for cell in row)


class UltimateTicTacToe:
    def __init__(self):
        self.boards = [[MiniBoard() for _ in range(3)] for _ in range(3)]
        self.main_winner = None
        self.current_player = PLAYER_X
        self.forced_board = None

    def make_move(self, board_row, board_col, cell_row, cell_col):
        if self.main_winner:
            return False

        if self.forced_board:
            r, c = self.forced_board
            board = self.boards[r][c]
            if board.winner or board.is_full():
                self.forced_board = None

        if self.forced_board and (board_row, board_col) != self.forced_board:
            return False

        board = self.boards[board_row][board_col]
        if board.make_move(cell_row, cell_col, self.current_player):
            self.check_main_winner()
            next_board = self.boards[cell_row][cell_col]
            if next_board.winner or next_board.is_full():
                self.forced_board = None
            else:
                self.forced_board = (cell_row, cell_col)
            self.current_player = PLAYER_O if self.current_player == PLAYER_X else PLAYER_X
            return True
        return False

    def check_main_winner(self):
        grid = [[self.boards[r][c].winner for c in range(3)] for r in range(3)]
        lines = grid + [list(col) for col in zip(*grid)]
        lines.append([grid[i][i] for i in range(3)])
        lines.append([grid[i][2 - i] for i in range(3)])
        for line in lines:
            if None not in line and 'Tie' not in line and len(set(line)) > 1:
                self.main_winner = line[0]
                return
        if all(self.boards[r][c].winner for r in range(3) for c in range(3)):
            self.main_winner = 'Tie'

    def get_valid_moves(self):
        moves = []
        if self.forced_board:
            r, c = self.forced_board
            board = self.boards[r][c]
            if not board.winner and not board.is_full():
                for i in range(3):
                    for j in range(3):
                        if board.board[i][j] == EMPTY:
                            moves.append((r, c, i, j))
        else:
            for br in range(3):
                for bc in range(3):
                    board = self.boards[br][bc]
                    if board.winner or board.is_full():
                        continue
                    for i in range(3):
                        for j in range(3):
                            if board.board[i][j] == EMPTY:
                                moves.append((br, bc, i, j))
        return moves

    def clone(self):
        return copy.deepcopy(self)


def minimax(game, depth, maximizing):
    if game.main_winner == PLAYER_X:
        return 10 - depth, None
    elif game.main_winner == PLAYER_O:
        return depth - 10, None
    elif game.main_winner == 'Tie':
        return 0, None

    if depth == 3:
        return 0, None

    best_move = None
    if maximizing:
        max_eval = float('-inf')
        for move in game.get_valid_moves():
            clone = game.clone()
            clone.make_move(*move)
            eval, _ = minimax(clone, depth + 1, False)
            if eval > max_eval:
                max_eval = eval
                best_move = move
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in game.get_valid_moves():
            clone = game.clone()
            clone.make_move(*move)
            eval, _ = minimax(clone, depth + 1, True)
            if eval < min_eval:
                min_eval = eval
                best_move = move
        return min_eval, best_move


class UltimateTicTacToeGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Ultimate Tic Tac Toe")

        self.title_frame = tk.Frame(self.root, width=600, height=600, bg='#d6eaf8')
        self.title_frame.pack_propagate(False)
        self.title_frame.pack(fill='both', expand=True)

        title = tk.Label(self.title_frame, text="\n\nUltimate Tic Tac Toe\n(Reversed Version)",
                         font=("Helvetica", 30, "bold"), fg="black", bg="#d6eaf8")
        title.pack(pady=20)

        members_text = (
            "Group Members:\n\n"
            "Ayesha Ansari 22K-4453\n"
            "Mishkaat Yousuf 22K-4624\n"
            "Jahantaab Kulsoom 22K-4214"
        )

        members_label = tk.Label(self.title_frame, text=members_text,
                                 font=("Helvetica", 16), fg="black", bg="#d6eaf8", justify="center")
        members_label.pack(pady=20)

        start_button = tk.Button(self.title_frame, text="Start Game", font=("Helvetica", 16, "bold"),
                                 command=self.start_game, bg="green", fg="white", width=15)
        start_button.pack(pady=40)

        self.canvas = None

    def start_game(self):
        self.title_frame.destroy()

        self.canvas = tk.Canvas(self.root, width=600, height=600, bg='white')
        self.canvas.pack()
        self.canvas.bind("<Button-1>", self.click)

        self.game = UltimateTicTacToe()
        self.draw_board()

    def draw_board(self):
        self.canvas.delete("all")
        for br in range(3):
            for bc in range(3):
                x0 = bc * 200
                y0 = br * 200
                mini = self.game.boards[br][bc]

                if mini.winner == PLAYER_X:
                    color = '#FF6B6B'
                elif mini.winner == PLAYER_O:
                    color = '#4DB6E2'
                elif mini.winner == 'Tie':
                    color = '#A9A9A9'
                elif (br, bc) == self.game.forced_board:
                    color = '#7fc5c0'
                else:
                    color = '#1C1C1C'

                self.canvas.create_rectangle(x0, y0, x0 + 200, y0 + 200, fill=color, outline='white')

                for i in range(3):
                    for j in range(3):
                        cell = mini.board[i][j]
                        cx = x0 + j * 65 + 33
                        cy = y0 + i * 65 + 33
                        self.canvas.create_rectangle(x0 + j * 65, y0 + i * 65,
                                                     x0 + (j + 1) * 65, y0 + (i + 1) * 65,
                                                     outline="white")
                        if cell:
                            self.canvas.create_text(cx, cy, text=cell,
                                                    font=("Arial", 20, "bold"),
                                                    fill="white")

        if self.game.main_winner:
            self.canvas.create_text(300, 300,
                                    text=f"Winner (Reversed): {self.game.main_winner}",
                                    font=("Arial", 30, "bold"),
                                    fill="white")

    def click(self, event):
        if self.game.main_winner:
            return
        row = event.y // 200
        col = event.x // 200
        cell_row = (event.y % 200) // 65
        cell_col = (event.x % 200) // 65

        moved = self.game.make_move(row, col, cell_row, cell_col)
        self.draw_board()
        if moved and self.game.current_player == PLAYER_O and not self.game.main_winner:
            self.root.after(300, self.ai_move)

    def ai_move(self):
        _, move = minimax(self.game, 0, True)
        if move:
            self.game.make_move(*move)
            self.draw_board()


if __name__ == '__main__':
    root = tk.Tk()
    app = UltimateTicTacToeGUI(root)
    root.mainloop()